In [36]:
%reload_ext autoreload
%autoreload 2

In [37]:
# Config dict to set the logging level
import logging.config
DEFAULT_LOGGING = {
    'version': 1,
    'disable_existing_loggers': False,
    'loggers': {
        '': {
            'level': 'INFO',
        },
    }
}

logging.config.dictConfig(DEFAULT_LOGGING)
# Disabling warnings output
import warnings
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
warnings.filterwarnings(action='ignore', category=UserWarning)

#!pip install dowhy
import dowhy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [38]:
df = pd.read_csv('loans_full_schema.csv')
# Drop irrelevant columns
df.drop(['emp_title', 'annual_income_joint', 'verification_income_joint', 
         'debt_to_income_joint', 'months_since_last_delinq', 'grade', 'sub_grade',
         'issue_month','tax_liens','loan_purpose','disbursement_method','paid_principal',
         'paid_interest','paid_late_fees','initial_listing_status','application_type',
         'verified_income','months_since_last_credit_inquiry',
         'num_collections_last_12m','num_historical_failed_to_pay','current_accounts_delinq',
         'current_installment_accounts','months_since_90d_late'], axis=1, inplace=True)
df.head()

,emp_length,state,homeownership,annual_income,debt_to_income,delinq_2y,earliest_credit_line,inquiries_last_12m,total_credit_lines,open_credit_lines,total_credit_limit,total_credit_utilized,total_collection_amount_ever,accounts_opened_24m,num_satisfactory_accounts,num_accounts_120d_past_due,num_accounts_30d_past_due,num_active_debit_accounts,total_debit_limit,num_total_cc_accounts,num_open_cc_accounts,num_cc_carrying_balance,num_mort_accounts,account_never_delinq_percent,public_record_bankrupt,loan_amount,term,interest_rate,installment,loan_status,balance,paid_total
0,3.0,NJ,MORTGAGE,90000.0,18.01,0,2001,6,28,10,70795,38767,1250,5,10,0.0,0,2,11100,14,8,6,1,92.9,0,28000,60,14.07,652.53,Current,27015.86,1999.33
1,10.0,HI,RENT,40000.0,5.04,0,1996,1,30,14,28800,4321,0,11,14,0.0,0,3,16500,24,14,4,0,100.0,1,5000,36,12.61,167.54,Current,4651.37,499.12
2,3.0,WI,RENT,40000.0,21.15,0,2006,4,31,10,24193,16000,432,13,10,0.0,0,3,4300,14,8,6,0,93.5,0,2000,36,17.09,71.40,Current,1824.63,281.80
3,1.0,PA,RENT,30000.0,10.16,0,2007,0,4,4,25400,4997,0,1,4,0.0,0,2,19400,3,3,2,0,100.0,0,21600,36,6.72,664.19,Current,18853.26,3312.89
4,10.0,CA,RENT,35000.0,57.96,0,2008,7,22,16,69839,52722,0,6,16,0.0,0,10,32700,20,15,13,0,100.0,0,23000,36,14.07,786.87,Current,21430.15,2324.65


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   emp_length                    9183 non-null   float64
 1   state                         10000 non-null  object 
 2   homeownership                 10000 non-null  object 
 3   annual_income                 10000 non-null  float64
 4   debt_to_income                9976 non-null   float64
 5   delinq_2y                     10000 non-null  int64  
 6   earliest_credit_line          10000 non-null  int64  
 7   inquiries_last_12m            10000 non-null  int64  
 8   total_credit_lines            10000 non-null  int64  
 9   open_credit_lines             10000 non-null  int64  
 10  total_credit_limit            10000 non-null  int64  
 11  total_credit_utilized         10000 non-null  int64  
 12  total_collection_amount_ever  10000 non-null  int64  
 13  ac

In [40]:
df["homeownership"].value_counts()

MORTGAGE    4789
RENT        3858
OWN         1353
Name: homeownership, dtype: int64

In [41]:
df["annual_income"].mean()

In [42]:
df["emp_length"].value_counts()

10.0    3332
2.0      967
3.0      862
0.0      690
1.0      685
5.0      645
4.0      611
6.0      404
7.0      368
9.0      312
8.0      307
Name: emp_length, dtype: int64

In [43]:
df['credit_line_utilization'] = round(df['open_credit_lines']/df['total_credit_lines']*100, 2)
df['credit_utilization'] = round(df['total_credit_utilized']/df['total_credit_limit']*100, 2)
df['num_accounts_past_due'] = df['num_accounts_120d_past_due'] + df['num_accounts_30d_past_due']
df['cc_utilization'] = round(df['num_open_cc_accounts'] / df['num_total_cc_accounts']*100, 2)



In [44]:
# if the customer ever has been deliquent

df['delinquent_history'] = np.where((df['account_never_delinq_percent'] == 100) | (df['delinq_2y'] == 0), True, False)

#how long the customer had credit for
df['credit_duration'] = 2021 - df['earliest_credit_line']

# credit inquires last 12 months
df['has_credit_inquiries_last_12m'] = np.where(df['inquiries_last_12m'] > 0, True, False)

# is sent to collection ever
df['has_collection_accounts'] = np.where(df['total_collection_amount_ever'] > 0, True, False)

# have accounts opened in last 24months
df['has_accounts_opened_24m'] = np.where(df['accounts_opened_24m'] > 0, True, False)

# has been bankrupt  ever
df['has_bankrupt_history'] = np.where(df['public_record_bankrupt'] > 0, True, False)

# has past due accounts
df['has_past_due_accounts'] = np.where(df['num_accounts_past_due'] > 0, True, False)

# has debt limit exceded
df['debt_limit_exceded'] = np.where(df['total_debit_limit'] - df['loan_amount'] > 0, False, True)

# is above avg income
df['not_low_income'] = np.where(df['annual_income'] > 35977, True, False)





In [45]:
df.tail(20)

,emp_length,state,homeownership,annual_income,debt_to_income,delinq_2y,earliest_credit_line,inquiries_last_12m,total_credit_lines,open_credit_lines,total_credit_limit,total_credit_utilized,total_collection_amount_ever,accounts_opened_24m,num_satisfactory_accounts,num_accounts_120d_past_due,num_accounts_30d_past_due,num_active_debit_accounts,total_debit_limit,num_total_cc_accounts,num_open_cc_accounts,num_cc_carrying_balance,num_mort_accounts,account_never_delinq_percent,public_record_bankrupt,loan_amount,term,interest_rate,installment,loan_status,balance,paid_total,credit_line_utilization,credit_utilization,num_accounts_past_due,cc_utilization,delinquent_history,credit_duration,has_credit_inquiries_last_12m,has_collection_accounts,has_accounts_opened_24m,has_bankrupt_history,has_past_due_accounts,debt_limit_exceded,not_low_income,good_employment_history
9980,2.0,MA,RENT,85000.0,12.40,0,2008,5,14,11,43846,20609,0,2,11,0.0,0,5,25600,10,10,7,0,100.0,0,15000,36,9.43,480.01,Current,13905.04,1432.17,78.57,47.00,0.0,100.00,True,13,True,False,True,False,False,False,True,False
9981,7.0,KY,MORTGAGE,64000.0,36.49,0,2002,1,37,14,434447,150679,0,5,14,0.0,0,4,26900,8,7,6,3,100.0,0,30000,36,9.93,967.03,In Grace Period,27089.00,3892.94,37.84,34.68,0.0,87.50,True,19,True,False,True,False,False,True,True,False
9982,0.0,CA,MORTGAGE,100000.0,21.71,0,1992,4,44,25,420116,81110,0,7,25,0.0,0,5,21900,24,16,10,4,100.0,1,16000,60,9.93,339.41,Current,15161.63,1348.81,56.82,19.31,0.0,66.67,True,29,True,False,True,True,False,False,True,False
9983,3.0,OH,MORTGAGE,114000.0,14.60,0,1987,0,41,19,217845,200883,330,8,19,0.0,0,7,44400,12,10,9,1,97.4,0,20000,36,9.93,644.69,Current,17564.07,3212.42,46.34,92.21,0.0,83.33,True,34,False,True,True,False,False,False,True,False
9984,7.0,CT,MORTGAGE,49000.0,36.20,0,2006,5,14,11,300238,47054,0,8,11,0.0,0,5,34300,7,7,7,1,100.0,0,31200,60,21.45,851.99,In Grace Period,29990.90,3333.60,78.57,15.67,0.0,100.00,True,15,True,False,True,False,False,False,True,False
9985,6.0,OH,MORTGAGE,106000.0,22.26,1,2004,1,16,10,387902,63727,0,3,10,0.0,0,2,15000,8,5,4,2,87.5,0,12500,36,7.35,387.97,Current,10923.77,1934.75,62.50,16.43,0.0,62.50,False,17,True,False,True,False,False,False,True,False
9986,0.0,NH,MORTGAGE,150000.0,6.26,0,2004,12,58,31,388580,33134,0,4,31,0.0,0,2,21700,27,21,8,1,100.0,0,40000,36,6.07,1218.15,Current,37963.23,2409.32,53.45,8.53,0.0,77.78,True,17,True,False,True,False,False,True,True,False
9987,6.0,TX,MORTGAGE,55000.0,22.19,0,2002,0,24,10,273702,57604,0,2,10,0.0,0,4,39000,10,7,5,3,100.0,0,16000,60,9.43,335.49,Current,15365.77,989.71,41.67,21.05,0.0,70.00,True,19,False,False,True,False,False,False,True,False
9988,1.0,MD,RENT,65000.0,9.77,0,2006,2,24,9,89497,99770,0,1,9,0.0,0,4,12100,9,7,7,0,100.0,0,3200,36,9.92,103.14,Current,2968.03,307.66,37.50,111.48,0.0,77.78,True,15,True,False,True,False,False,False,True,False
9989,10.0,AZ,RENT,65000.0,27.10,0,1982,1,26,22,87123,56137,0,6,22,0.0,0,6,18400,20,17,10,0,92.3,0,10000,60,15.05,238.17,Current,9421.91,1174.13,84.62,64.43,0.0,85.00,True,39,True,False,True,False,False,False,True,True


In [46]:
# set bad loan/good loan
bad_loan = ["Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off", "In Grace Period", 
            "Late (16-30 days)", "Late (31-120 days)"]
df['is_bad_loan'] = np.where(df['loan_status'].isin(bad_loan), True, False)


# set homeownership
ownership = ['MORTGAGE','OWN']
df['is_homeowner'] = np.where(df['homeownership'].isin(ownership), True, False)

In [47]:
df.to_csv('loans_causal_schema.csv', index=False)

In [48]:
df.info()
print(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 48 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   emp_length                     9183 non-null   float64
 1   state                          10000 non-null  object 
 2   homeownership                  10000 non-null  object 
 3   annual_income                  10000 non-null  float64
 4   debt_to_income                 9976 non-null   float64
 5   delinq_2y                      10000 non-null  int64  
 6   earliest_credit_line           10000 non-null  int64  
 7   inquiries_last_12m             10000 non-null  int64  
 8   total_credit_lines             10000 non-null  int64  
 9   open_credit_lines              10000 non-null  int64  
 10  total_credit_limit             10000 non-null  int64  
 11  total_credit_utilized          10000 non-null  int64  
 12  total_collection_amount_ever   10000 non-null  

In [49]:
df["emp_length"].value_counts()

10.0    3332
2.0      967
3.0      862
0.0      690
1.0      685
5.0      645
4.0      611
6.0      404
7.0      368
9.0      312
8.0      307
Name: emp_length, dtype: int64